In [4]:
#multi channel CNN for sentiment analysis
from nltk.corpus import stopwords
from string import punctuation,digits
import pandas as pd
import json
import numpy as np
import re
#import word2vecReader as godin_embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook as tqdm
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
import torch
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
from keras.layers.merge import concatenate
import keras.backend as K
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [5,5]
plt.style.use('seaborn-notebook')
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error,r2_score
#from aspect_specific_prob import get_normalized_sentence_relation_vector
from math import sqrt
#from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
import nltk
nltk.download('stopwords')
import json
from embedding_as_service.text.encode import Encoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/menglong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Load Training and Testing Data 
with open('/home/menglong/workspace/code/multimodal/mocheg/SpotFakePlus/data/train/news_article.json', 'r') as f:
    testData = json.load(f)

In [6]:
embeddings = {}
lengths = []

In [9]:
# XLNET Model from Hugging Face
from  transformers import XLNetModel, XLNetTokenizer
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import threading
os.environ["CUDA_VISIBLE_DEVICES"]="1"
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')
model.to('cuda')

XLNetModel(
  (word_embedding): Embedding(32000, 768)
  (layer): ModuleList(
    (0): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, bias=True)
        (layer_2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, b

In [10]:
# Generate embeddings for a paragraph
def torchEmbeddings(data):
    embed = []
    sentences= sent_tokenize(data)
    for sent in sentences:
        input_ids = torch.tensor(tokenizer.encode(sent)).unsqueeze(0)
        input_ids = input_ids.to('cuda')
        tempEmbedding = model(input_ids)
        #temp=en.encode([sent],pooling='reduce_mean')
        embed.append(tempEmbedding)
    return embed

In [11]:
embeddings

{}

In [12]:
# Dump Embeddings into a pickle file
import pickle
with open('data/feature/finalTrainEmbeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

### Some optimizations for large Embedding FIles

In [17]:
queryList = [ testData ]

In [16]:
list(set(testData))

['title',
 'canonical_link',
 'news_source',
 'authors',
 'url',
 'movies',
 'summary',
 'images',
 'keywords',
 'source',
 'text',
 'meta_data',
 'top_img',
 'publish_date']

In [18]:
exceptions = []

In [19]:
import pickle
"""with open('gossip/finalTrainEmbeddings1.pkl', 'wb') as f:
    pickle.dump(embeddings, f)"""

"with open('gossip/finalTrainEmbeddings1.pkl', 'wb') as f:\n    pickle.dump(embeddings, f)"

In [24]:
from tqdm.notebook import tqdm
with torch.no_grad():
    embeddings = {}
    for i in tqdm(queryList):
        try:
            embeddings[i] = torchEmbeddings(i["text"])
        except:
            exceptions.append(i)
    with open('data/feature/finalTestEmbeddings2.pkl', 'wb') as f:
        pickle.dump(embeddings, f)
print(embeddings)
            #print(len(sent_tokenize(i)))
            #lengths.append(len(sent_tokenize(i)))

  0%|          | 0/1 [00:00<?, ?it/s]

{}


In [ ]:
with open('imageDataset/politi/testJson.json', 'r') as f:
    testData = json.load(f)

In [ ]:
testEmbeddings = {}
with torch.no_grad():
    for i in tqdm(testData):
        testEmbeddings[i] = torchEmbeddings(i)
        #lengths.append(len(sent_tokenize(i)))

In [ ]:
# For politifact
import pickle
with open('politifact/finalTrainEmbeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)
with open('politifact/finalTestEmbeddings.pkl', 'wb') as f:
    pickle.dump(testEmbeddings, f)